# Hysteresis: Calculate the area inside the mean seasonal cycles

In [ ]:
include("stuff.jl")

In [ ]:
pca_msc_cube = loadCube(joinpath(cubes_base_dir, "pca_msc_cube"))

In [ ]:
rmprocs(procs())
addprocs(40)
@everywhere begin
    """
    Area inside a polygon, will be positive if anticlockwise, 
    negative if clockwise, self intersecting polygons will cancel out (partially?).
    """
    function area(x::Vector{T}, y::Vector{T})::T where T
        @assert length(x) == length(y)

        a = zero(T)
        @inbounds for i in 2:(length(x) - 1)
            a += x[i] * (y[i + 1] - y[i - 1])
        end
        a += x[1] * (y[2] - y[end])
        a /= 2
        return a
    end
end
    
cube_hysteresis_12 = mapCube(
    pca_msc_cube,
    indims = InDims("PcaAxis", "MSC"),
    outdims = OutDims()
) do xout, xin
    
    x = xin[1, :]
    y = xin[2, :]
    
    if any(ismissing, x) || any(ismissing, y)
        xout[1] = missing
    else
        xout[1] = area(x, y)
    end    
end

cube_hysteresis_13 = mapCube(
    pca_msc_cube,
    indims = InDims("PcaAxis", "MSC"),
    outdims = OutDims()
) do xout, xin
    
    x = xin[1, :]
    y = xin[3, :]
    
    if any(ismissing, x) || any(ismissing, y)
        xout[1] = missing
    else
        xout[1] = area(x, y)
    end    
end

cube_hysteresis_23 = mapCube(
    pca_msc_cube,
    indims = InDims("PcaAxis", "MSC"),
    outdims = OutDims()
) do xout, xin
    
    x = xin[2, :]
    y = xin[3, :]
    
    if any(ismissing, x) || any(ismissing, y)
        xout[1] = missing
    else
        xout[1] = area(x, y)
    end    
end

rmprocs(procs())

In [ ]:
R"""
library(viridis)
data(coastsCoarse, package = "rworldmap")

pdf($(joinpath(fig_path, "hysteresis.pdf")), width = 10*2/3, height = 10)
layout(matrix(c(1:3), ncol = 1))

pal1 <- c("#543005", "#8C510A", "#BF812D", "#DFC27D", 
          "#F6E8C3", "#F5F5F5", "#C7EAE5", "#80CDC1", 
          "#35978F", "#01665E", "#003C30")
pal1 <- colorRampPalette(colors = pal1)(100)

pal2 <- rev(c("#67001f", "#b2182b","#d6604d", "#f4a582",
              "#fddbc7", "#f7f7f7", "#d1e5f0","#92c5de",
              "#4393c3", "#2166ac","#053061"))
pal2 <- colorRampPalette(colors = pal2)(100)

pal3 <- c("#7f3b08","#b35806","#e08214","#fdb863",
          "#fee0b6","#f7f7f7","#d8daeb","#b2abd2",
          "#8073ac","#542788","#2d004b")
pal3 <- colorRampPalette(colors = pal3)(100)

hyst12 <- $(cube_hysteresis_12[:,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])
hyst13 <- $(cube_hysteresis_13[:,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])
hyst23 <- $(cube_hysteresis_23[:,:] |> collect |> missing_to_nan |> x -> x[:, end:-1:1])

crange12 <- 0.8 * (max(abs(range(hyst12, na.rm = TRUE))) %>% { c(-., .) })
crange13 <- 0.8 * (max(abs(range(hyst13, na.rm = TRUE))) %>% { c(-., .) })
crange23 <- 0.8 * (max(abs(range(hyst23, na.rm = TRUE))) %>% { c(-., .) })

hist12 <- hist(hyst12, plot = FALSE)
hist13 <- hist(hyst13, plot = FALSE)
hist23 <- hist(hyst23, plot = FALSE)

############ maps
par(mar = c(0, 0, 0, 0))
image(seq(-180, 180, by = 0.25), seq(-60, 90, by = 0.25), hyst12[, 120:720], zlim = crange12, col = pal1, useRaster = TRUE, bty = 'n', axes = FALSE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-60, 90, by = 0.25), hyst13[, 120:720], zlim = crange13, col = pal2, useRaster = TRUE, bty = 'n', axes = FALSE)
plot(coastsCoarse, add = TRUE)
image(seq(-180, 180, by = 0.25), seq(-60, 90, by = 0.25), hyst23[, 120:720], zlim = crange23, col = pal3, useRaster = TRUE, bty = 'n', axes = FALSE)
plot(coastsCoarse, add = TRUE)

############ color bars and hists
par(fig = c(0.07, 0.10, (2 + 0.05) / 3, (2 + 0.5) / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange12[1], crange12[2], length.out = 100),
      t(seq(crange12[1], crange12[2], length.out = 100)), 
      col = pal1, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      #ylab = expression(atop("Area " * group("[", "std" ^ 2, "]"),
      #                       "clockwise"%<->% "counterclockwise")),
      ylab = expression("Area " * group("[", "std" ^ 2, "]")),
      xlab = NA
)
text(1, crange12[1], "clockwise", pos = 4)
text(1, crange12[2], "counterclockwise", pos = 4)
rect(1, hist12$breaks[-length(hist12$breaks)], 1 + 4 * hist12$density, hist12$breaks[-1])


par(fig = c(0.07, 0.10, (1 + 0.05) / 3, (1 + 0.5) / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange13[1], crange13[2], length.out = 100),
      t(seq(crange13[1], crange13[2], length.out = 100)), 
      col = pal2, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      #ylab = expression(atop("Area " * group("[", "std" ^ 2, "]"),
      #                       "clockwise"%<->% "counterclockwise")),
      ylab = expression("Area " * group("[", "std" ^ 2, "]")),
      xlab = NA
)
text(1, crange13[1], "clockwise", pos = 4)
text(1, crange13[2], "counterclockwise", pos = 4)
rect(1, hist13$breaks[-length(hist13$breaks)], 1 + 4 * hist13$density, hist13$breaks[-1])

par(fig = c(0.07, 0.10, 0.05 / 3, 0.5 / 3), new = TRUE, xpd = NA, mgp = c(2.2, 1, 0))
image(0:1, 
      seq(crange23[1], crange23[2], length.out = 100),
      t(seq(crange23[1], crange23[2], length.out = 100)), 
      col = pal3, useRaster = TRUE, xaxt = 'n', bty = 'n', las = 2, 
      #ylab = expression(atop("Area " * group("[", "std" ^ 2, "]"),
      #                       "clockwise"%<->% "counterclockwise")),
      ylab = expression("Area " * group("[", "std" ^ 2, "]")),
      xlab = NA
)
text(1, crange23[1], "clockwise", pos = 4)
text(1, crange23[2], "counterclockwise", pos = 4)
rect(1, hist23$breaks[-length(hist23$breaks)], 1 + 4 * hist23$density, hist23$breaks[-1])

######## decoration
grid.text("(a)", 0,   1, just = c(0, 1))
grid.text("(b)", 0, 2/3, just = c(0, 1))
grid.text("(c)", 0, 1/3, just = c(0, 1))


dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/hysteresis.pdf", width = 700, height = 1000)

In [ ]:
pca_msc_cube

In [ ]:
R"""
pdf($(joinpath(fig_path, "area_example.pdf")), width = 7, height = 7 * 11/20)
par(mar = c(4, 4, 0, 0) + 0.1)
plot(c(-10, 10), c(-5, 6), type = "n", xlab = "x", ylab = "y", bty = "n")

area_poly <- function (x, y) {
    n <- length(x)

    a <- x[1] * (y[2] - y[n])
    for (i in 2:(n - 1)) {
        a <- a + x[i] * (y[i + 1] - y[i - 1])
    }
    a <- a + x[n] * (y[1] - y[n - 1])
    a <- a / 2

    return(a)
}

plot_poly <- function (x, y, xshift = 0, yshift = 0, label = "", arrow_pos = 15) {
    xx <- x + xshift
    yy <- y + yshift
    xx <- c(xx, xx[1])
    yy <- c(yy, yy[1])
    a <- sprintf("%0.2f", area_poly(x, y))
    lines(xx, yy, lwd = 2)
    text(min(xx) - 0.5, max(yy) + 0.5, paste(label, "Area = ", a))
    if (!is.na(arrow_pos))
        arrows(xx[arrow_pos], yy[arrow_pos], xx[arrow_pos + 1], yy[arrow_pos + 1], length = 0.1, lwd = 2)
}
"""


# somewhere in east asia with positive hysteresis 12
lat, lon = 42.1390871, 125.0747612

compx = 1
compy = 2

loni = axVal2Index(getAxis(LonAxis, pca_msc_cube), lon)
lati = axVal2Index(getAxis(LatAxis, pca_msc_cube), lat)

mscx = pca_msc_cube[:, compx, loni, lati] .|> Float64
mscy = pca_msc_cube[:, compy, loni, lati] .|> Float64

R"""
plot_poly($mscx, $mscy, -5, 2.5, "(a)")
"""

# spain with positive hysteresis 13
lat, lon = 39.2894848, -5.9888035

compx = 1
compy = 3

loni = axVal2Index(getAxis(LonAxis, pca_msc_cube), lon)
lati = axVal2Index(getAxis(LatAxis, pca_msc_cube), lat)

mscx = pca_msc_cube[:, compx, loni, lati] .|> Float64
mscy = pca_msc_cube[:, compy, loni, lati] .|> Float64

R"""
plot_poly($mscx, $mscy, 5, 5, "(b)", 20)
"""

#india monsoon
lat, lon = 22.375, 82.375
compx = 1
compy = 2

loni = axVal2Index(getAxis(LonAxis, pca_msc_cube), lon)
lati = axVal2Index(getAxis(LatAxis, pca_msc_cube), lat)

mscx = pca_msc_cube[:, compx, loni, lati] .|> Float64
mscy = pca_msc_cube[:, compy, loni, lati] .|> Float64

R"""
plot_poly($mscx, $mscy, 5, -2.5, "(d)", 1)
"""

# amazon
lat, lon = -2.625, -65.625
compx = 1
compy = 2

loni = axVal2Index(getAxis(LonAxis, pca_msc_cube), lon)
lati = axVal2Index(getAxis(LatAxis, pca_msc_cube), lat)

mscx = pca_msc_cube[:, compx, loni, lati] .|> Float64
mscy = pca_msc_cube[:, compy, loni, lati] .|> Float64

R"""
plot_poly($mscx, $mscy, -7, -2.5, "(c)", NA)
dev.off()
"""

show_pdf("http://127.0.0.1:8333/files/results/fig/area_example.pdf", width = 700, height = 500)